# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [50]:
def print_table(labels, cell_data):
    fig=plt.figure()
    nrows, ncols = len(cell_data)+1, len(labels)
    hcell, wcell = 10, 2
    #\hpad, wpad = 0.5, 0.5    
    #fig=plt.figure(figsize=(ncols*wcell+wpad, nrows*hcell+hpad))
    ax = fig.add_subplot(111)
    ax.axis('off')
    the_table = ax.table(cellText=cell_data,
              colLabels=labels,loc='left')
    table_props = the_table.properties()
    table_cells = table_props['child_artists']
    for cell in table_cells: cell.set_height(0.4)
    the_table.set_fontsize(14)
    #the_table.scale(2,2)
    plt.show()
    
def P1(num_examples=5):
    ### STUDENT START ###
    try:
        num_examples = int(num_examples)
        if(num_examples == 0):
            print "Please enter number of samples to be greater than 0"
            raise 
        width = 200
        for i in range(num_examples):
            
             print 'Example: %3.0f Text Message:%s Text Label: %s' %(i+1,train_data[i],newsgroups_train.target_names[train_labels[i]])
    except Exception as e:
        print "Exception Occured: ", e.args
        raise
    
### STUDENT END ###
#P1(5)
data = [(train_data[0],train_labels[0]),(train_data[1],train_labels[1])]
print_table(("Text", "Label"),data )

NameError: name 'textwrap' is not defined

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [56]:
def P2():
    ### STUDENT START ###
    try:
        ##### 2.a
        vectorizer = CountVectorizer(min_df=1)
        X = vectorizer.fit_transform(train_data)
        print 'Answer to 2.a :'
        print 'Size of vocabulary: ', X.shape[1]
        print 'Average number of non-zero features per example: ', round(1.0 * X.nnz/X.shape[0],3)
        print 'Fraction of entries in the matrix that are non zero: ', round(1.0 * X.nnz/ (X.shape[1] * X.shape[0]),3)
        
        ###### 2.b
        feature_names = vectorizer.get_feature_names()
        #print 'Feature set:',feature_names
        #feature_names.sort()
        print '\nAnswer to 2.b :'
        print '0th feture: %s  and last feature: %s' %(feature_names[0], feature_names[len(feature_names)-1])
        
        ###### 2.c
        # setting up the custome vocabulary
        cust_vocab = ['atheism', 'religion', 'graphics', 'space']
        # initialise the Count vectorizer with the custom vocabulary
        cust_vectorizer = CountVectorizer(min_df=1,vocabulary = cust_vocab )
        X = cust_vectorizer.fit_transform(train_data)
        print '\nAnswer to 2.c:'
        print 'Size of vocabulary with custom vocabulary:',X.shape[1]
        print 'Average number of non-zero feaures per example: ', round(1.0 * X.nnz/X.shape[0],3)
        
        ###### 2.d
        bigram_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2,3), min_df=1)
        bi_X = bigram_vectorizer.fit_transform(train_data)
        print '\nAnswer to 2.d :'
        print 'Size of vocabulary for bigram & trigram: ', bi_X.shape[1]
        #print bigram_vectorizer.vocabulary_
        
        
        
        ###### 2.e
        min_vectorizer = CountVectorizer(min_df=10)
        X = min_vectorizer.fit_transform(train_data)
        print '\nAnswer to 2.e :'
        print 'Size of vocabulary with Min-df = 10 : ', X.shape[1]
        
        ###### 2.f
        train_vectorizer,dev_vectorizer1 = CountVectorizer(encoding='utf8'),CountVectorizer(encoding='utf8')
        X_train = train_vectorizer.fit(train_data)
        X_dev = dev_vectorizer1.fit(dev_data)
        print '\nAnswer to 2.f : '
        # Determine how many features are common between training and dev set
        common =  len(list(set(X_train.get_feature_names()) & set(X_dev.get_feature_names())))
        print 'Number of features common between training and dev set: ', common
        print "Fraction missing ", round(1.0*(len(X_dev.get_feature_names())-common)/len(X_dev.get_feature_names()),3)

        
    except Exception as e:
        print "Exception Occured: ", e.args
        raise

### STUDENT END ###
P2()

Answer to 2.a :
Size of vocabulary:  26879
Average number of non-zero features per example:  96.706
Fraction of entries in the matrix that are non zero:  0.004

Answer to 2.b :
0th feture: 00  and last feature: zyxel

Answer to 2.c:
Size of vocabulary with custom vocabulary: 4
Average number of non-zero feaures per example:  0.268

Answer to 2.d :
Size of vocabulary for bigram & trigram:  35478
(2034, 35478)

Answer to 2.e :
Size of vocabulary with Min-df = 10 :  3064

Answer to 2.f : 
Number of features common between training and dev set:  12219
Fraction missing  0.248


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [103]:
def P3():
    ### STUDENT START ###
    try:
        #default CountVectorizer
        def_vectorizer = CountVectorizer()
        x_train = def_vectorizer.fit_transform(train_data)
        x_dev   = def_vectorizer.transform(dev_data)
        x_test =  def_vectorizer.transform(test_data)
        k_values = {'n_neighbors':[1,3,5,7,9]}
        knn = KNeighborsClassifier()
        
        
        pipeline = Pipeline([
                #( 'clf', LogisticRegression()),
                ('multiNB' , MultinomialNB()),
                ('kNN', KNeighborsClassifier())   
            ])

        parameters = {
        #'clf__C': (1, 10),
        #'clf__penalty': ('l1', 'l2'),
        'multiNB__alpha':(1,0.1,0.01,0.001,0.0001,0),
        'kNN__n_neighbors': (1,5,7,9,15,20,25,20,30,40,50,60,70,80,90,100,150,200,250,300),
            }

        
        # Create and Initialise GridSearch  and fit on the binarized training data
        grid_search = GridSearchCV(pipeline, parameters,refit=true)
        grid_search.fit(x_train,train_labels)
        
        print("\nPerforming grid search...")
        print("pipeline:", [name for name, _ in pipeline.steps])
        print("parameters:")
        pprint(parameters)

        print "Best Parameter found on Training data: ", grid_search.best_estimator_ 
        print("Grid scores on Mini Train set:")
        for params, mean_score, scores in grid_search.grid_scores_:
            print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params))
        
        print("\nBest score: {:0.3f}".format(grid_search.best_score_))
        print("\nBest parameters set:")
        best_parameters = grid_search.best_estimator_.get_params()
        for param_name in sorted(parameters.keys()):
            print("\t{}: {}".format(param_name, best_parameters[param_name]))
        
        y_true, y_pred = dev_labels, grid_search.predict(x_dev)
        print"f1 score: ",(metrics.f1_score(y_true, y_pred))
        

        
    except Exception as e:
        print "Exception Occured: ", e.args
        raise
        

    ### STUDENT END ###
P3()

Exception Occured:  ("All intermediate steps a the chain should be transforms and implement fit and transform 'MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)' (type <class 'sklearn.naive_bayes.MultinomialNB'>) doesn't)",)


TypeError: All intermediate steps a the chain should be transforms and implement fit and transform 'MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)' (type <class 'sklearn.naive_bayes.MultinomialNB'>) doesn't)

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [5]:
#def P4():
### STUDENT START ###

### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [6]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [15]:
#def P6():
    # Keep this random seed here to make comparison easier.
    #np.random.seed(0)

    ### STUDENT START ###
    
    ### STUDENT END ###
#P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [16]:
#def P7():
### STUDENT START ###

## STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

In [93]:
# extra space
vectorizer = CountVectorizer(min_df=1,analyzer='word',token_pattern='(?u)\\b\\w+\\b')
#vectorizer = CountVectorizer(min_df=1,analyzer='char',ngram_range=(1,3))
# corpus = ['This is the first document.',
#           'This is the second second document.',
#           'And the third one.',
#           'I Is this the first document?',
#          ]
corpus = train_data[7]
print corpus
X = vectorizer.fit_transform(corpus)
print X.shape
print vectorizer.get_feature_names()


Acorn Replay running on a 25MHz ARM 3 processor (the ARM 3 is about 20% slower
than the ARM 6) does this in software (off a standard CD-ROM). 16 bit colour at
about the same resolution (so what if the computer only has 8 bit colour
support, real-time dithering too...). The 3D0/O is supposed to have a couple of
DSPs - the ARM being used for housekeeping.


A 25MHz ARM 6xx should clock around 20 ARM MIPS, say 18 flat out. Depends
really on the surrounding system and whether you are talking ARM6x or ARM6xx
(the latter has a cache, and so is essential to run at this kind of speed with
slower memory).

I'll stop saying things there 'cos I'll hopefully be working for ARM after
graduation...

Mike

PS Don't pay heed to what reps from Philips say; if the 3D0/O doesn't beat the
   pants off 3DI then I'll eat this postscript.
(828, 31)
[u'0', u'1', u'2', u'3', u'5', u'6', u'8', u'a', u'b', u'c', u'd', u'e', u'f', u'g', u'h', u'i', u'k', u'l', u'm', u'n', u'o', u'p', u'r', u's', u't', u'u', u'v'

In [86]:
#np.random.randint(low=1,high=200, size=15)
np.array([1,0.1,0.01,0.001,0.0001,0])

array([  1.00000000e+00,   1.00000000e-01,   1.00000000e-02,
         1.00000000e-03,   1.00000000e-04,   0.00000000e+00])